In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# linear regression

# y = wX + b   => wX      w: D+1 dimension
# use MSE/MAE   
# L = ((y-y^hat))^2;   y^hat = wX
# dL/dw = 1/N * 2(y - y_hat)*x, 
# w = w - r*dL/dw


Pros:

Linear regression is straightforward to understand and explain, and can be regularized to avoid overfitting. In addition, linear models can be updated easily with new data using stochastic gradient descent.


Cons:

Linear regression performs poorly when there are non-linear relationships. They are not naturally flexible enough to capture more complex patterns, and adding the right interaction terms or polynomials can be tricky and time-consuming.

In [ ]:
import numpy as np
from numpy.random import rand
from sklearn import datasets
from sklearn.metrics import precision_recall_fscore_support

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from sklearn.model_selection import train_test_split
from copy import deepcopy

In [ ]:
class LinearRegression(object):

    def __init__(self):
        self.w = None          #dimension D x 1         
        self.loss = []
        self.history_weights = []
        
    def linear_func(self, X, w):
        return np.dot(X, w)
    
    def fit(self, X, y, epochs=20, lr=0.05):
        # input: X -- NXD ; y -- NX1
        # gradient descent
                
        w = rand(X.shape[1])   # weight: Dx1
        b = rand(X.shape[1])   
        
        N = len(X)          # number of instances
        for i in range(epochs):
            
            #Gradient descent
            y_hat = self.linear_func(X, w)
            #print("ss11111: ", X.shape, y.shape, w.shape, b.shape, y_hat.shape)
           
            dw = 2 * np.dot(X.T, y_hat - y)/N
            
            w -= lr * dw        # lr * dot(X.T, y_hat -y)/ N
            
            curr_loss = self.loss_func(X, y, w)
            #print("wwwww: ", w)
            self.loss.append(curr_loss)
            
            self.history_weights.append(deepcopy(w))
            
        self.w = w
        self.b = b
                                   
    def loss_func(self, X, y_true, w):
        # use MSE here
        N = len(X)
        y_hat = self.linear_func(X, w)
        loss = 1/N*(np.sum(np.square((y_true - y_hat))))
        #print("loss: ", loss)

        return loss
    
    def predict(self, X):
        return self.linear_func(X, self.w)

Read data to train and test

In [ ]:
# read data;  generate random data
n_samples = 100
X, y = datasets.make_regression(n_samples=n_samples, n_features=1,
                                      n_informative=1, noise=10,
                                      coef=False, random_state=0)

print ("X:", X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



In [ ]:
# train the model
LinearRegression_obj = LinearRegression()
epochs = 50
LinearRegression_obj.fit(X_train, y_train, epochs=epochs)

print("LinearRegression_obj.history_weights: ", LinearRegression_obj.history_weights)


In [ ]:
# test
y_test_hat = LinearRegression_obj.predict(X_test)


In [ ]:
# plot
plt.plot(LinearRegression_obj.loss)
plt.show()
#print("y: ", y)
#print("y_test_hat: ", y_test_hat)
print("loss: ", LinearRegression_obj.loss)
    

In [ ]:
# Animation

#Set the plot up,
fig = plt.figure()
ax = plt.axes()
plt.title('Sale Price vs Living Area')
plt.xlabel('Living Area in square feet (normalised)')
plt.ylabel('Sale Price ($)')
plt.scatter(X_train, y_train, color='red')
line, = ax.plot([], [], lw=2)
annotation = ax.text(-1, 700000, '')
annotation.set_animated(True)
plt.close()

#Generate the animation data,
def init():
    line.set_data([], [])
    annotation.set_text('')
    return line, annotation

# animation function.  This is called sequentially
def animate(i):
    x = np.linspace(-5, 20, 100)
    #print("LinearRegression_obj.history_weights[i]: ", LinearRegression_obj.history_weights[i])
    y = LinearRegression_obj.history_weights[i][0]*x
    line.set_data(x, y)
    annotation.set_text('Cost = %.2f e10' % (LinearRegression_obj.loss[i]/10000000000))
    return line, annotation

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=epochs, interval=0, blit=True)

anim.save('animation.gif', writer='imagemagick', fps = 30)

In [ ]:
#Display the animation...
import io
import base64
from IPython.display import HTML

filename = 'animation.gif'

video = io.open(filename, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii')))